In [ ]:
!pip install --upgrade -q torch
!pip install --upgrade -q tensorflow
!pip install --upgrade -q jax
!pip install --upgrade -q keras-nlp
!pip install --upgrade -q keras

# Some care is required to install Keras 3. This is a temporary situation.
# See installation notes at the end of this notebook for details.

In [1]:
#@title Backend selection and display utilities [run me]
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def big_print(a,b):
  html = '<div style="font-size: 18pt; font-family: monospace">{}{}</div>'.format(a, b)
  display(HTML(html))
def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")

backend = 'tensorflow' # @param ["jax", "tensorflow", "torch"]

In [2]:
import math, os, random
os.environ['KERAS_BACKEND'] = backend

import keras
import keras_nlp

backend = keras.config.backend()
big_print('\u2B50 ', 'Keras version '+keras.version())
big_print('\u2B50 ', 'Running on '+backend.upper())

<hr/>

# <img src="https://keras.io/img/k-logo.png" height="80pt" align="center"/> Keras 3: Let us checkout a generative model and build a chatbot - OPT causal

OPT is a causal language model, it continues the input prompt.

In [5]:
# model
keras.utils.set_random_seed(42)
nlp_model = keras_nlp.models.OPTCausalLM.from_preset("opt_125m_en")
nlp_model.compile(sampler=keras_nlp.samplers.ContrastiveSampler())

In [7]:
prompt = "Hi, I'm a {} machine learning developer. \
          What are you working on?".format(backend.upper())
response = nlp_model.generate(prompt, max_length=57)
response = response.replace(prompt, '')
big_print("\U0001F64B ",prompt)
big_print("\U0001F916 ",response)